In [ ]:
def connect(dbname,password,host="localhost",user="root",port=3306):
    import MySQLdb as mc 
    db = mc.connect(host=host,user=user,passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,newsTime=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    data = (category,title,newsDate,newsTime,journalist,content,hit,url,source)
    cur.execute('''insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values("%s","%s","%s","%s","%s","%s","%s","%s","%s")''',data)
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    x = (commentDate,commentTime,commenter,commenterUrl,comment,url)
    cur.execute('''insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values("%s","%s","%s","%s","%s","%s")''' ,x)
    db.commit()
def insert_newsRaw(db,category,url,newsRaw,source):
    cur = db.cursor()
    t = (category,url,newsRaw,source)
    cur.execute('''insert into newsRaw(`class`,url,newsRaw,source)values("%s","%s","%s","%s")''',t)
    db.commit()
def close(db):
    db.close()
import requests
from bs4 import BeautifulSoup
import time
import re
import traceback
import csv
loginurl="http://udndata.com/ndapp/member/MbFixLogin?708869834"
headers={
'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'Accept-Encoding':'gzip, deflate, sdch',
'Accept-Language':'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4',
'Connection':'keep-alive',
'Host':'udndata.com',
'Referer':'http://udndata.com/library/',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'
}


with open('xae.csv', 'rb') as csvfile:
	rows = csv.reader(csvfile, delimiter='\n', quotechar='\t')
	for r in rows:
		try:
			f=open("ecoerror.txt","a")
			fc=open("collect.csv","a")
			row=r[0].split("\t")[1]
			category=r[0].split("\t")[0]
			#print category, row
			rs = requests.session()
			rs.get(loginurl,headers=headers)
			res=rs.get(row).text
			conn=connect(dbname="ecodaily3",password="ho",host="192.168.196.159",user="ho",port=3306)
			try :
				insert_newsRaw(conn,category,row,res,"Economoic Daily")
				fc.write(category+"\t"+row+"\n")
				close(conn)
			except:
				f.write('error: '+row+"\n")
				f.write( traceback.format_exc())
		except:
			f.write('error: '+row+"\n")
			f.write( traceback.format_exc())
			time.sleep(5)
	close(f)
	close(fc)